本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [5]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split, GridSearchCV # 划分训练集和测试集
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # 评价指标
RANDOM_SEED = 2023 # 固定随机种子

### 读取数据和预处理

In [6]:
import os
import numpy as np

data_dir = os.path.join('.', 'data/trec06c-utf8') # 数据集路径
label_file = os.path.join(data_dir, 'label', 'index') # 标签路径

labels_array = [] # 存储标签
texts_array = [] # 存储文本

with open(label_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        label, path = line.strip().split()
        if label == 'spam':
            labels_array.append(1)
        else:
            labels_array.append(0)
        with open(os.path.join(data_dir, path.replace('../data', 'data_cut')), 'r', encoding='utf-8') as email_f:
            email_lines = email_f.readlines()
            is_body = False
            email_text = []
            for email_line in email_lines:
                email_line = email_line.strip()
                if email_line == '':
                    is_body = True
                if is_body:
                    email_text += email_line.split()
            texts_array.append(' '.join(email_text))

labels = np.array(labels_array)

### 划分数据集

In [7]:
# 划分训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(texts_array, labels, test_size=0.2, random_state=RANDOM_SEED)

# 划分训练集和验证集的坐标
train_indices, val_indices = train_test_split(np.arange(len(train_texts)), test_size=0.1, random_state=RANDOM_SEED)

### 定义分类器模型

In [8]:
from typing import Union, Dict

class NaiveBayes:
    def __init__(self, max_df: Union[float, int] = 1.0, min_df: Union[float, int] = 1, tfidf: bool = False, type: str = 'multinomial') -> None:
        '''
        初始化
        :param max_df: 最大文档频率，超过该阈值的词语会被过滤掉
        :param min_df: 最小文档频率，低于该阈值的词语会被过滤掉
        :param tfidf: 是否使用tfidf
        :param type: 朴素贝叶斯算法类型，可选'multinomial', 'bernoulli', 'complement'
        '''
        self.max_df = max_df
        self.min_df = min_df
        self.tfidf = tfidf
        self.type = type
    
    def fit(self, X, y) -> None:
        '''
        训练模型
        :param X: 训练集特征
        :param y: 训练集标签
        :return: None
        '''
        if self.tfidf:
            self.vectorizer = TfidfVectorizer(max_df=self.max_df, min_df=self.min_df)
        else:
            self.vectorizer = CountVectorizer(max_df=self.max_df, min_df=self.min_df)
        X = self.vectorizer.fit_transform(X)
        if self.type == 'multinomial':
            self.model = MultinomialNB()
        elif self.type == 'bernoulli':
            self.model = BernoulliNB()
        elif self.type == 'complement':
            self.model = ComplementNB()
        else:
            raise ValueError('Unknown type: {}'.format(self.type))
        self.model.fit(X, y)
    
    def predict(self, X) -> np.ndarray:
        '''
        预测
        :param X: 测试集特征
        :return: 测试集标签
        '''
        assert hasattr(self, 'vectorizer'), 'Please train the model first!'
        assert hasattr(self, 'model'), 'Please train the model first!'
        X = self.vectorizer.transform(X)
        return self.model.predict(X)
    
    def get_params(self, deep: bool = True) -> Dict[str, Union[float, int, bool, str]]:
        '''
        获取参数
        :param deep: 是否深拷贝
        :return: 参数
        '''
        return {
            'max_df': self.max_df,
            'min_df': self.min_df,
            'tfidf': self.tfidf,
            'type': self.type
        }
    
    def set_params(self, **params) -> 'NaiveBayes':
        '''
        设置参数
        :param params: 参数
        :return: None
        '''
        for param, value in params.items():
            setattr(self, param, value)
        return self

### 训练和测试模型

In [9]:
model = NaiveBayes(type='multinomial')
model.fit(train_texts, train_labels)
pred_test_labels = model.predict(test_texts)
print(f'Params: {model.get_params()}, Accuracy: {accuracy_score(test_labels, pred_test_labels)}, Precision: {precision_score(test_labels, pred_test_labels)}, Recall: {recall_score(test_labels, pred_test_labels)}, F1: {f1_score(test_labels, pred_test_labels)}')

Params: {'max_df': 1.0, 'min_df': 1, 'tfidf': False, 'type': 'multinomial'}, Accuracy: 0.9779480037140205, Precision: 0.9814899977006208, Recall: 0.9856829465419698, F1: 0.9835820035716344


In [10]:
params = {
    'max_df': [1.0, 0.999, 0.998, 0.997, 0.996, 0.995, 0.99],
    'min_df': range(1, 7),
    'tfidf': [True, False],
    'type': ['multinomial', 'bernoulli', 'complement']
}
grid_search = GridSearchCV(NaiveBayes(), params, cv=[(train_indices, val_indices)], scoring='f1', verbose=100, n_jobs=-1)
grid_search.fit(train_texts, train_labels)
print(f'Best params: {grid_search.best_params_}, Best score: {grid_search.best_score_}')

best_model = NaiveBayes(**grid_search.best_params_)
best_model.fit(train_texts, train_labels)
pred_test_labels = best_model.predict(test_texts)
print(f'Params: {best_model.get_params()}, Accuracy: {accuracy_score(test_labels, pred_test_labels)}, Precision: {precision_score(test_labels, pred_test_labels)}, Recall: {recall_score(test_labels, pred_test_labels)}, F1: {f1_score(test_labels, pred_test_labels)}')

Fitting 1 folds for each of 252 candidates, totalling 252 fits
Best params: {'max_df': 1.0, 'min_df': 2, 'tfidf': True, 'type': 'multinomial'}, Best score: 0.9864411721825338
Params: {'max_df': 1.0, 'min_df': 2, 'tfidf': True, 'type': 'multinomial'}, Accuracy: 0.9802692664809657, Precision: 0.9796850034238759, Recall: 0.9911095716429973, F1: 0.9853641737932617


### 小结

在本数据集及特征环境下，朴素贝叶斯文本分类的多项式模型 MultinomialNB 和补足模型 ComplementNB 表现较好，他们考虑了词出现的频率，伯努利模型只考虑是否出现。提取的两个头信息特征对效果有略微提升。文本向量化的两种方式次数向量 CountVectorizer 和 tf-idf 向量 TfidfVectorizer 与不同模型结合有略微差别。